# 🎵 Building the Hot-100 Lyrics Dataset

**Purpose:**  
Pull unique song lyrics for every track in the 2024 Billboard Hot-100, cache raw JSON per song, handle “not found” cases, and assemble a master table.

**Outline:**  
1. Load weekly chart → extract unique songs  
2. Initialize Genius client & cache folder  
3. Fetch & cache raw JSON (Genius + fallbacks)  
4. Merge caches into a Parquet master table  
5. Save for downstream NLP/graph work  

*Run each code cell in order, tweak parameters as needed, and rerun from the top if you change anything in the pipeline.*

In [36]:
# ─── Imports & Configuration ───

# Load environment variables from .env (for GENIUS_TOKEN)
from dotenv import load_dotenv
import os
load_dotenv()

# Standard libraries
import time
import json
import pathlib

# Third-party
import pandas as pd
import lyricsgenius as lg

# ─── Paths & Constants ───
RAW_EXCEL  = "../data/raw/billboard/billboard_dataset_2024.xlsx"
CACHE_DIR  = "../data/raw/lyrics"

In [2]:
# ─── Helper: Unique Song ID ───
def make_song_id(title: str, artist: str) -> str:
    """
    Create a filesystem-safe, unique ID for each song based on artist + title.
    """
    slug = f"{artist}__{title}".lower().replace("/", "_").replace(" ", "_")
    # keep only letters, numbers, and underscores
    return "".join(ch for ch in slug if ch.isalnum() or ch == "_")

In [86]:
# ─── Load Chart & Extract Unique Songs ───

# 1️⃣ Read the raw Billboard Hot-100 Excel into a DataFrame
df_chart = pd.read_excel(RAW_EXCEL)

# 2️⃣ Inspect the columns and a sample row to confirm names
print("Columns:", df_chart.columns.tolist())
print(df_chart.head(2))

# 3️⃣ Create a unique song identifier (without week)
df_chart["song_id"] = df_chart.apply(
    lambda row: make_song_id(row["artist"], row["title"]),
    axis=1
)

# 4️⃣ Build a DataFrame of unique songs to fetch
df_songs = (
    df_chart[["song_id", "title", "artist"]]
    .drop_duplicates()
    .reset_index(drop=True)
)

# 5️⃣ Quick stats and preview
print(f"Total weekly entries: {len(df_chart)}")
print(f"Unique songs to fetch: {len(df_songs)}")
df_songs.head(5)

Columns: ['date', 'rank', 'title', 'artist', 'image', 'peakPos', 'lastpos', 'weeks', 'isNew']
         date  rank                              title        artist  \
0  2024-12-28     1    All I Want For Christmas Is You  Mariah Carey   
1  2024-12-28     2  Rockin' Around The Christmas Tree    Brenda Lee   

                                               image  peakPos  lastpos  weeks  \
0  https://charts-static.billboard.com/img/1994/1...        1        1     70   
1  https://charts-static.billboard.com/img/1960/1...        1        2     63   

   isNew  
0  False  
1  False  
Total weekly entries: 5200
Unique songs to fetch: 761


,song_id,title,artist
0,all_i_want_for_christmas_is_you__mariah_carey,All I Want For Christmas Is You,Mariah Carey
1,rockin_around_the_christmas_tree__brenda_lee,Rockin' Around The Christmas Tree,Brenda Lee
2,last_christmas__wham,Last Christmas,Wham!
3,jingle_bell_rock__bobby_helms,Jingle Bell Rock,Bobby Helms
4,a_holly_jolly_christmas__burl_ives,A Holly Jolly Christmas,Burl Ives


In [87]:
# Save to Excel
df_chart.to_excel(
    "../data/raw/billboard/billboard_dataset_2024_with_id.xlsx",     # output file path (use “.xls” or “.xlsx”)
    sheet_name="2024",# optional: name of the worksheet
    index=False         # don’t write row indices into the file
)


In [10]:
# ─── Initialize Genius Client ───

# 1️⃣ Grab your Genius API token from the environment
token = os.getenv("TOKEN")
assert token, "🔑 TOKEN not found—make sure you set it in your .env"

# 2️⃣ Create the client
genius = lg.Genius(
    token,
    timeout=10,               # seconds to wait per request
    retries=3,                # how many times to retry on failure
    skip_non_songs=True,      # skip instrumentals, podcasts, etc.
    remove_section_headers=True  # strips [Chorus], [Verse], etc.
)

print("✅ Genius client initialized successfully!")

✅ Genius client initialized successfully!


In [37]:
# ─── Test Fetch for a Single Song ───

# 1️⃣ Select the first unique song for testing
test_row = df_songs.iloc[0]
song_id = test_row["song_id"]
title   = test_row["title"]
artist  = test_row["artist"]

print(f"""Testing fetch for:
 • song_id: {song_id}
 • Title:   {title}
 • Artist:  {artist}
""")


# 3️⃣ Fetch and cache if not already present
try:
    song = genius.search_song(title, artist)
    song.save_lyrics(filename=f"{CACHE_DIR}/{song_id}.json", sanitize=False, overwrite=True)
    print("✅ Lyrics found!")
except Exception as e:
    # Handle errors from the API call or missing data
    print(f"❌ Error fetching lyrics: {e}")

Testing fetch for:
 • song_id: all_i_want_for_christmas_is_you__mariah_carey
 • Title:   All I Want For Christmas Is You
 • Artist:  Mariah Carey

Searching for "All I Want For Christmas Is You" by Mariah Carey...
Done.
Wrote ../data/raw/lyrics/all_i_want_for_christmas_is_you__mariah_carey.json.
✅ Lyrics found!


In [39]:
# ─── Bulk Fetch All Unique Song Lyrics ───

# Loop over all unique songs and fetch lyrics (no 'exists' check)
results = []  # collect summary of fetch status
for idx, row in df_songs.iterrows():
    song_id = row["song_id"]
    title   = row["title"]
    artist  = row["artist"]
    cache_path = f"{CACHE_DIR}/{song_id}.json"

    # Prepare result record
    try:
        song = genius.search_song(title, artist)
        song.save_lyrics(filename=f"{CACHE_DIR}/{song_id}.json", sanitize=False, overwrite=True)
        print("✅ Lyrics found!")
    except Exception as e:
        print(f"⚠️ Failed to fetch {song_id}: {e}")
    
print(f"🎉 Completed fetching songs.")

Searching for "All I Want For Christmas Is You" by Mariah Carey...
Done.
Wrote ../data/raw/lyrics/all_i_want_for_christmas_is_you__mariah_carey.json.
✅ Lyrics found!
Searching for "Rockin' Around The Christmas Tree" by Brenda Lee...
Done.
Wrote ../data/raw/lyrics/rockin_around_the_christmas_tree__brenda_lee.json.
✅ Lyrics found!
Searching for "Last Christmas" by Wham!...
Done.
Wrote ../data/raw/lyrics/last_christmas__wham.json.
✅ Lyrics found!
Searching for "Jingle Bell Rock" by Bobby Helms...
Done.
Wrote ../data/raw/lyrics/jingle_bell_rock__bobby_helms.json.
✅ Lyrics found!
Searching for "A Holly Jolly Christmas" by Burl Ives...
Done.
Wrote ../data/raw/lyrics/a_holly_jolly_christmas__burl_ives.json.
✅ Lyrics found!
Searching for "Die With A Smile" by Lady Gaga & Bruno Mars...
Done.
Wrote ../data/raw/lyrics/die_with_a_smile__lady_gaga__bruno_mars.json.
✅ Lyrics found!
Searching for "It's The Most Wonderful Time Of The Year" by Andy Williams...
Done.
Wrote ../data/raw/lyrics/its_the_mos

In [85]:
# ─── List Missing Songs ───
import os

# 1️⃣ List all cached song IDs by inspecting the cache directory (string path)
fnames = [f for f in os.listdir(CACHE_DIR) if f.endswith('.json')]
cached_song_ids = [os.path.splitext(f)[0] for f in fnames]

# 2️⃣ Identify which songs in df_songs are not present in the cache
missing = [sid for sid in df_songs['song_id'] if sid not in cached_song_ids]

# 3️⃣ Build DataFrame of missing entries
df_missing = df_songs[df_songs['song_id'].isin(missing)].reset_index(drop=True)

# 4️⃣ Display results
print(f"Songs missing lyrics cache: {len(df_missing)}")
df_missing

Songs missing lyrics cache: 0


,song_id,title,artist
